In [1]:
# Import the TensorFlow and output the verion

!pip install tensorflow==1.14.0
import tensorflow as tf
print("\n\nTensorFlow version:", tf.__version__)

/home/caesar/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/caesar/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/caesar/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/caesar/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dty



TensorFlow version: 1.14.0


/home/caesar/anaconda3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/caesar/anaconda3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/caesar/anaconda3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/caesar/anaconda3/lib/python3.7/site-packages/tensorboar

In [2]:
n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

In [3]:
tf.reset_default_graph()
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")

In [4]:
# By default, the tf.layers.dense() function uses Xavier initialization (with uniform distribution)

hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, name="hidden1")

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [5]:
# You can change this to He initialization by using the variance_scaling_initializer() function like this:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
he_init = tf.contrib.layers.variance_scaling_initializer()
hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu,
                         kernel_initializer=he_init, name="hidden1")

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [6]:
# TensorFlow offers an elu() function that you can use to build your neural network. Simply set the activation 
# argument when calling the dense() function like this:

tf.reset_default_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.elu, name="hidden1")

In [7]:
# TensorFlow does not have a predefined function for leaky ReLUs, but it is easy to define:

tf.reset_default_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
def leaky_relu(z, name=None):
    return tf.maximum(0.01 * z, z, name=name)

hidden1 = tf.layers.dense(X, n_hidden1, activation=leaky_relu, name="hidden1")

# Implementing Batch Normalization with TensorFlow
Another way to reduce the problem of vanishing/exploding gradients is to use **batch normalization**.
TensorFlow provides a **tf.nn.batch_normalization()** function that simply centers and normalizes the inputs, but 
 you must compute the mean and standard deviation yourself and pass them as parameters to this function, and you must also handle the creation of the scaling and offset parameters (and pass them to this function). Another option is to use the **tf.layers.batch_normalization()** function, which handles all this for you as in the following code:

In [8]:
tf.reset_default_graph()

n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")

training = tf.placeholder_with_default(False, shape=(), name="training")

hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1")
bn1 = tf.layers.batch_normalization(hidden1, training=training, momentum=0.9)
bn1_act = tf.nn.elu(bn1)
hidden2 = tf.layers.dense(bn1_act, n_hidden2, name="hidden2")
bn2 = tf.layers.batch_normalization(hidden2, training=training, momentum=0.9)
bn2_act = tf.nn.elu(bn2)
logits_before_bn = tf.layers.dense(bn2_act, n_outputs, name="outputs")
logits = tf.layers.batch_normalization(logits_before_bn, training=training, momentum=0.9)

Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).


###### The code is quite repetitive, with the same batch normalization parameters appearing over and over again. To avoid this repetition, you can use the partial() function from the functools module. It creates a thin wrapper aroud a function and allows you to define default values for some parameters. The creation of the network layers in the preceding code can be modified as follows:

In [9]:
tf.reset_default_graph()

from functools import partial

n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")

training = tf.placeholder_with_default(False, shape=(), name="training")

my_batch_norm_layer = partial(tf.layers.batch_normalization, training=training, momentum=0.9)

hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1")
bn1 = my_batch_norm_layer(hidden1)
bn1_act = tf.nn.elu(bn1)
hidden2 = tf.layers.dense(bn1_act, n_hidden2, name="hidden2")
bn2 = my_batch_norm_layer(hidden2)
bn2_act = tf.nn.elu(bn2)
logits_before_bn = tf.layers.dense(bn2_act, n_outputs, name="outputs")
logits = my_batch_norm_layer(logits_before_bn)

#### Let's build a neural net for MNIST, using the ELU activation function and Batch Normalization at each layer:

In [10]:
tf.reset_default_graph()

from functools import partial

batch_norm_momentum = 0.9
learning_rate = 0.01

n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")
training = tf.placeholder_with_default(False, shape=(), name="training")

with tf.name_scope("dnn"):
    he_init = tf.contrib.layers.variance_scaling_initializer()
    my_batch_norm_layer = partial(tf.layers.batch_normalization, training=training, momentum=batch_norm_momentum)
    my_dense_layer = partial(tf.layers.dense, kernel_initializer=he_init)
    
    hidden1 = my_dense_layer(X, n_hidden1, name="hidden1")
    bn1 = tf.nn.elu(my_batch_norm_layer(hidden1))
    hidden2 = my_dense_layer(bn1, n_hidden2, name="hidden2")
    bn2 = tf.nn.elu(my_batch_norm_layer(hidden2))
    logits_before_bn = my_dense_layer(bn2, n_outputs, name="outputs")
    logits = my_batch_norm_layer(logits_before_bn)
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
    
with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [11]:
n_epochs = 40
batch_size = 64

In [6]:
import numpy as np

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

In [13]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [14]:
extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run([training_op, extra_update_ops], feed_dict={training: True, X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Validation accuracy:", accuracy_val)
    
    save_path = saver.save(sess, "./my_model_final.ckpt")

0 Validation accuracy: 0.9104
1 Validation accuracy: 0.932
2 Validation accuracy: 0.9442
3 Validation accuracy: 0.9544
4 Validation accuracy: 0.9582
5 Validation accuracy: 0.9608
6 Validation accuracy: 0.964
7 Validation accuracy: 0.9672
8 Validation accuracy: 0.9688
9 Validation accuracy: 0.97
10 Validation accuracy: 0.9726
11 Validation accuracy: 0.972
12 Validation accuracy: 0.9714
13 Validation accuracy: 0.9724
14 Validation accuracy: 0.9732
15 Validation accuracy: 0.9746
16 Validation accuracy: 0.9746
17 Validation accuracy: 0.9754
18 Validation accuracy: 0.9754
19 Validation accuracy: 0.9766
20 Validation accuracy: 0.9756
21 Validation accuracy: 0.9746
22 Validation accuracy: 0.9762
23 Validation accuracy: 0.9768
24 Validation accuracy: 0.9782
25 Validation accuracy: 0.978
26 Validation accuracy: 0.977
27 Validation accuracy: 0.9772
28 Validation accuracy: 0.9776
29 Validation accuracy: 0.9776
30 Validation accuracy: 0.9794
31 Validation accuracy: 0.978
32 Validation accuracy: 0.

# Gradient Clipping
A popular technique to lessen the exploding gradients problem is to simply clip the gradients during backpropagation so they never exceed some threshold. This is called **Gradient Clipping**. In TensorFlow, the optimizer's minimize() function takes care of both computing the gradients and applying them, so you must instead call the optimizer's **compute_gradients()** methods first, then create an operation to clip the gradients using the **clip_by_value()** function, and finally create an operation to apply the clipped gradients using the optimizer's **apply_gradients()** method:

In [31]:
tf.reset_default_graph()

n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 50
n_hidden3 = 50
n_hidden4 = 50
n_hidden5 = 50
n_outputs = 10



X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=None, name= "y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu, name="hidden2")
    hidden3 = tf.layers.dense(hidden2, n_hidden3, activation=tf.nn.relu, name="hidden3")
    hidden4 = tf.layers.dense(hidden3, n_hidden4, activation=tf.nn.relu, name="hidden4")
    hidden5 = tf.layers.dense(hidden4, n_hidden5, activation=tf.nn.relu, name="hidden5")
    logits = tf.layers.dense(hidden5, n_outputs, name="outputs")
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

In [32]:
learning_rate = 0.01
threshold = 1.0

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    grads_and_vars = optimizer.compute_gradients(loss)
    capped_gvs = [(tf.clip_by_value(grad, -threshold, threshold), var) for grad, var in grads_and_vars]
    training_op = optimizer.apply_gradients(capped_gvs)
    
# It will compute the gradients, clip them between -1.0 and 1.0, and apply them.

In [33]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [34]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [35]:
n_epochs = 40
batch_size = 64

In [36]:
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Validation accuracy:", accuracy_val)
        

    save_path = saver.save(sess, "./my_model_final.ckpt")

0 Validation accuracy: 0.8574
1 Validation accuracy: 0.912
2 Validation accuracy: 0.9326
3 Validation accuracy: 0.9436
4 Validation accuracy: 0.9542
5 Validation accuracy: 0.96
6 Validation accuracy: 0.9644
7 Validation accuracy: 0.966
8 Validation accuracy: 0.9682
9 Validation accuracy: 0.969
10 Validation accuracy: 0.9678
11 Validation accuracy: 0.9724
12 Validation accuracy: 0.9728
13 Validation accuracy: 0.9754
14 Validation accuracy: 0.9734
15 Validation accuracy: 0.9752
16 Validation accuracy: 0.975
17 Validation accuracy: 0.9742
18 Validation accuracy: 0.9722
19 Validation accuracy: 0.9758
20 Validation accuracy: 0.9752
21 Validation accuracy: 0.9764
22 Validation accuracy: 0.9782
23 Validation accuracy: 0.9778
24 Validation accuracy: 0.9756
25 Validation accuracy: 0.9756
26 Validation accuracy: 0.9764
27 Validation accuracy: 0.9744
28 Validation accuracy: 0.9774
29 Validation accuracy: 0.9738
30 Validation accuracy: 0.977
31 Validation accuracy: 0.9768
32 Validation accuracy: 0

# Reusing Pretrained Layers
## Reusing a TensorFlow Model
If the original model was trained using TensorFlow, you can simply restore it and train it on the new task. You can use the **import_meta_graph()** function to import the operations into the default graph. This returns a **Saver** that you can use later to load the model's state (i.e., the variable values):

In [37]:
tf.reset_default_graph()

saver = tf.train.import_meta_graph("./my_model_final.ckpt.meta")

In [38]:
# To list all the operations you can use the graph's get_operation() method:

for op in tf.get_default_graph().get_operations():
    print(op.name)

X
y
hidden1/kernel/Initializer/random_uniform/shape
hidden1/kernel/Initializer/random_uniform/min
hidden1/kernel/Initializer/random_uniform/max
hidden1/kernel/Initializer/random_uniform/RandomUniform
hidden1/kernel/Initializer/random_uniform/sub
hidden1/kernel/Initializer/random_uniform/mul
hidden1/kernel/Initializer/random_uniform
hidden1/kernel
hidden1/kernel/Assign
hidden1/kernel/read
hidden1/bias/Initializer/zeros
hidden1/bias
hidden1/bias/Assign
hidden1/bias/read
dnn/hidden1/MatMul
dnn/hidden1/BiasAdd
dnn/hidden1/Relu
hidden2/kernel/Initializer/random_uniform/shape
hidden2/kernel/Initializer/random_uniform/min
hidden2/kernel/Initializer/random_uniform/max
hidden2/kernel/Initializer/random_uniform/RandomUniform
hidden2/kernel/Initializer/random_uniform/sub
hidden2/kernel/Initializer/random_uniform/mul
hidden2/kernel/Initializer/random_uniform
hidden2/kernel
hidden2/kernel/Assign
hidden2/kernel/read
hidden2/bias/Initializer/zeros
hidden2/bias
hidden2/bias/Assign
hidden2/bias/read
dn

In [39]:
# Once you know which operations you need, you can get a handle on them  using the graph's get_operation_by_name()
# and get_tensor_by_name() methods:

X = tf.get_default_graph().get_tensor_by_name("X:0")
y = tf.get_default_graph().get_tensor_by_name("y:0")
accuracy = tf.get_default_graph().get_tensor_by_name("eval/accuracy:0")
training_op = tf.get_default_graph().get_operation_by_name("train/GradientDescent")

**Note:** Name of tensor is the name of the operation that outputs it followed by :0(or :1 if it is the second output, :2 if it is the third, and so on).

In [40]:
# If you are the author of the original model, you could make things easier for people who will reuse your model
# by giving operations very clear names and documenting them. Another approach is to create a collection containing
# all the important operations that will want to get a handle on:

for op in (X, y, accuracy, training_op):
    tf.add_to_collection("my_important_ops", op)

In [41]:
# This way people who will reuse your model will be able to simple write:

X, y, accuracy, training_op = tf.get_collection("my_important_ops")

In [42]:
# You can then restore the model's state using the Saver and continue training using your own data:


with tf.Session() as sess:
    saver.restore(sess, "./my_model_final.ckpt")

    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Validation accuracy:", accuracy_val)

    save_path = saver.save(sess, "./my_new_model_final.ckpt")

INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt
0 Validation accuracy: 0.9776
1 Validation accuracy: 0.9782
2 Validation accuracy: 0.9786
3 Validation accuracy: 0.9798
4 Validation accuracy: 0.9784
5 Validation accuracy: 0.9792
6 Validation accuracy: 0.9788
7 Validation accuracy: 0.9786
8 Validation accuracy: 0.9786
9 Validation accuracy: 0.9788
10 Validation accuracy: 0.979
11 Validation accuracy: 0.9798
12 Validation accuracy: 0.9792
13 Validation accuracy: 0.9794
14 Validation accuracy: 0.9792
15 Validation accuracy: 0.9784
16 Validation accuracy: 0.9792
17 Validation accuracy: 0.9794
18 Validation accuracy: 0.9784
19 Validation accuracy: 0.979
20 Validation accuracy: 0.9784
21 Validation accuracy: 0.9798
22 Validation accuracy: 0.9788
23 Validation accuracy: 0.9794
24 Validation accuracy: 0.9784
25 Validation accuracy: 0.9792
26 Validation accuracy: 0.9792
27 Validation accuracy: 0.9792
28 Validation accuracy: 0.9792
29 Validation accuracy: 0.9788
30 Validation accu

In general you will want to reuse only the lower layers. If you are using **import_meta_graph()** it will load the whole graph, but you can simply ignore the parts you do not need. In this example, we add a new 4th hidden layer on top of the pretrained 3rd layer (ignoring the old 4th hidden layer). We also build a new output layer, the loss for this new output, and a new optimizer to minimize it. We also need another saver to save the whole graph (containing both the entire old graph plus the new operations), and an initialization operation to initialize all the new variables:

In [43]:
tf.reset_default_graph()

n_hidden4 = 20 
n_outputs = 10

saver = tf.train.import_meta_graph("./my_model_final.ckpt.meta")

X = tf.get_default_graph().get_tensor_by_name("X:0")
y = tf.get_default_graph().get_tensor_by_name("y:0")

hidden3 = tf.get_default_graph().get_tensor_by_name("dnn/hidden3/Relu:0")

new_hidden4 = tf.layers.dense(hidden3, n_hidden4, activation=tf.nn.relu, name="new_hidden4")
new_logits = tf.layers.dense(new_hidden4, n_outputs, name="new_outputs")

with tf.name_scope("new_loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=new_logits)
    loss = tf.reduce_mean(xentropy, name="loss")
    
with tf.name_scope("new_eval"):
    correct = tf.nn.in_top_k(new_logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

with tf.name_scope("new_train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()
new_saver = tf.train.Saver()

In [44]:
# And we can train this model:
with tf.Session() as sess:
    init.run()
    saver.restore(sess, "./my_model_final.ckpt")
    
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Validation accuracy:", accuracy_val)
    
    save_path = new_saver.save(sess, "./my_new_model_final.ckpt")

INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt
0 Validation accuracy: 0.9552
1 Validation accuracy: 0.9686
2 Validation accuracy: 0.9744
3 Validation accuracy: 0.9766
4 Validation accuracy: 0.9772
5 Validation accuracy: 0.978
6 Validation accuracy: 0.9786
7 Validation accuracy: 0.9782
8 Validation accuracy: 0.9774
9 Validation accuracy: 0.979
10 Validation accuracy: 0.9806
11 Validation accuracy: 0.9784
12 Validation accuracy: 0.9784
13 Validation accuracy: 0.9806
14 Validation accuracy: 0.979
15 Validation accuracy: 0.9798
16 Validation accuracy: 0.9806
17 Validation accuracy: 0.9804
18 Validation accuracy: 0.9794
19 Validation accuracy: 0.9802
20 Validation accuracy: 0.9808
21 Validation accuracy: 0.98
22 Validation accuracy: 0.9788
23 Validation accuracy: 0.9798
24 Validation accuracy: 0.9794
25 Validation accuracy: 0.9792
26 Validation accuracy: 0.98
27 Validation accuracy: 0.98
28 Validation accuracy: 0.9798
29 Validation accuracy: 0.979
30 Validation accuracy: 0.

If you have access to the Python code that built the original graph, you can just reuse the parts you need and drop the rest:

In [45]:
tf.reset_default_graph()

n_inputs = 28 * 28
n_hidden1 = 300 # reused
n_hidden2 = 50  # reused
n_hidden3 = 50  # reused
n_hidden4 = 20  # new
n_outputs = 10  # new

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu, name="hidden2")
    hidden3 = tf.layers.dense(hidden2, n_hidden3, activation=tf.nn.relu, name="hidden3")
    hidden4 = tf.layers.dense(hidden3, n_hidden4, activation=tf.nn.relu, name="hidden4")
    logits = tf.layers.dense(hidden4, n_outputs, name="outputs")
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")
    
with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

However, you must create one **Saver** to restore the pretrained model (giving it the list of variables to restore, or else it will complain that the graphs don't match), and another **Saver** to save the new model, once it is trained:

In [46]:
reuse_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope="hidden[123]")
restore_saver = tf.train.Saver(reuse_vars) # To restore layers 1-3

init = tf.global_variables_initializer() # To init all variables, old and new
saver = tf.train.Saver()

with tf.Session() as sess:
    init.run()
    restore_saver.restore(sess, "./my_model_final.ckpt")
    
    for epoch in range(n_epochs):                                            
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size): 
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})        
        accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})     
        print(epoch, "Validation accuracy:", accuracy_val)    
    
    save_path = saver.save(sess, "./my_new_model_final.ckpt")

INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt
0 Validation accuracy: 0.9614
1 Validation accuracy: 0.9694
2 Validation accuracy: 0.972
3 Validation accuracy: 0.9748
4 Validation accuracy: 0.9728
5 Validation accuracy: 0.974
6 Validation accuracy: 0.9776
7 Validation accuracy: 0.9768
8 Validation accuracy: 0.9778
9 Validation accuracy: 0.9792
10 Validation accuracy: 0.9792
11 Validation accuracy: 0.9794
12 Validation accuracy: 0.98
13 Validation accuracy: 0.9796
14 Validation accuracy: 0.9786
15 Validation accuracy: 0.98
16 Validation accuracy: 0.9794
17 Validation accuracy: 0.9804
18 Validation accuracy: 0.9804
19 Validation accuracy: 0.9778
20 Validation accuracy: 0.9808
21 Validation accuracy: 0.9754
22 Validation accuracy: 0.9802
23 Validation accuracy: 0.9806
24 Validation accuracy: 0.98
25 Validation accuracy: 0.9802
26 Validation accuracy: 0.9808
27 Validation accuracy: 0.9802
28 Validation accuracy: 0.98
29 Validation accuracy: 0.9804
30 Validation accuracy: 0.

# Reusing Models from Other Frameworks
If the model was trained using another framework, you will need to load the model parameters manually, then assign them to the appropriate variables.

In [47]:
tf.reset_default_graph()

n_inputs = 2
n_hidden1 = 3

original_W = [[1., 2., 3.], [4., 5., 6.]]
original_b = [7., 8., 9.]

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, name="hidden1")
#  [...] Build the rest of the model

# Get a handle on the assignment nodes for the hidden1 variables
graph = tf.get_default_graph()
assign_kernel = graph.get_operation_by_name("hidden1/kernel/Assign")
assign_bias = graph.get_operation_by_name("hidden1/bias/Assign")
init_kernel = assign_kernel.inputs[1]
init_bias = assign_bias.inputs[1]

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init, feed_dict={init_kernel: original_W, init_bias: original_b})
    # [...] Train the model on your new task
    print(hidden1.eval(feed_dict={X: [[10.0, 11.0]]}))
    
# the weights variable created by the tf.layers.dense() function is called "kernel" (instead of "weights" when 
# using the tf.contrib.layers.fully_connected(), as in the book), and the biases variable is called bias instead 
# of biases.

[[ 61.  83. 105.]]


# Freezing the Lower Layers
It is likely that the lower layers of the first DNN have learned to detect low-level features up in pictures that will be useful across both image classification tasks, so you can just reuse these layers as they are. It is generally a good idea to "freeze" their weights when training the new DNN: if the lower-layer weights are fixed, then the higher-layer weights will be easier to train. 

In [50]:
# To freeze the lower layers during training, one solution is to give the optimizer the lost of variables to 
# train, excluding the variables from the lower layers:

tf.reset_default_graph()

n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 50
n_hidden3 = 50
n_hidden4 = 20
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=None, name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu, name="hidden2")
    hidden3 = tf.layers.dense(hidden2, n_hidden3, activation=tf.nn.relu, name="hidden3")
    hidden4 = tf.layers.dense(hidden3, n_hidden4, activation=tf.nn.relu, name="hidden4")
    logits = tf.layers.dense(hidden4, n_outputs, name="outputs")
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

with tf.name_scope("train"):                                        
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)    
    train_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="hidden[34]|outputs")
    training_op = optimizer.minimize(loss, var_list=train_vars)    

reuse_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,
                               scope="hidden[123]") # regular expression
restore_saver = tf.train.Saver(reuse_vars) # to restore layers 1-3

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    init.run()
    restore_saver.restore(sess, "./my_model_final.ckpt")

    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Validation accuracy:", accuracy_val)

    save_path = saver.save(sess, "./my_new_model_final.ckpt")

INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt
0 Validation accuracy: 0.9632
1 Validation accuracy: 0.9698
2 Validation accuracy: 0.9724
3 Validation accuracy: 0.9728
4 Validation accuracy: 0.974
5 Validation accuracy: 0.9748
6 Validation accuracy: 0.9748
7 Validation accuracy: 0.974
8 Validation accuracy: 0.9758
9 Validation accuracy: 0.9754
10 Validation accuracy: 0.9764
11 Validation accuracy: 0.9764
12 Validation accuracy: 0.9774
13 Validation accuracy: 0.9768
14 Validation accuracy: 0.9778
15 Validation accuracy: 0.9774
16 Validation accuracy: 0.9772
17 Validation accuracy: 0.9774
18 Validation accuracy: 0.9786
19 Validation accuracy: 0.9788
20 Validation accuracy: 0.9782
21 Validation accuracy: 0.9788
22 Validation accuracy: 0.9784
23 Validation accuracy: 0.9782
24 Validation accuracy: 0.9782
25 Validation accuracy: 0.9784
26 Validation accuracy: 0.9786
27 Validation accuracy: 0.9792
28 Validation accuracy: 0.9792
29 Validation accuracy: 0.9788
30 Validation accu

In [51]:
# Another option is to add a stop_gradient() layer in the graph. Any layer below it will be frozen:

tf.reset_default_graph()

n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 50
n_hidden3 = 50
n_hidden4 = 20
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=None, name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu, name="hidden2")
    hidden2_stop = tf.stop_gradient(hidden2)
    hidden3 = tf.layers.dense(hidden2_stop, n_hidden3, activation=tf.nn.relu, name="hidden3")
    hidden4 = tf.layers.dense(hidden3, n_hidden4, activation=tf.nn.relu, name="hidden4")
    logits = tf.layers.dense(hidden4, n_outputs, name="outputs")
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

reuse_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope="hidden[123]") 
restore_saver = tf.train.Saver(reuse_vars) # to restore layers 1-3

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    init.run()
    restore_saver.restore(sess, "./my_model_final.ckpt")

    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Validation accuracy:", accuracy_val)

    save_path = saver.save(sess, "./my_new_model_final.ckpt")

INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt
0 Validation accuracy: 0.96
1 Validation accuracy: 0.9682
2 Validation accuracy: 0.9702
3 Validation accuracy: 0.9716
4 Validation accuracy: 0.9726
5 Validation accuracy: 0.9738
6 Validation accuracy: 0.9742
7 Validation accuracy: 0.9746
8 Validation accuracy: 0.9754
9 Validation accuracy: 0.9746
10 Validation accuracy: 0.9752
11 Validation accuracy: 0.9756
12 Validation accuracy: 0.976
13 Validation accuracy: 0.9756
14 Validation accuracy: 0.9768
15 Validation accuracy: 0.9758
16 Validation accuracy: 0.9766
17 Validation accuracy: 0.9768
18 Validation accuracy: 0.9768
19 Validation accuracy: 0.9756
20 Validation accuracy: 0.9768
21 Validation accuracy: 0.9772
22 Validation accuracy: 0.9768
23 Validation accuracy: 0.9772
24 Validation accuracy: 0.9772
25 Validation accuracy: 0.977
26 Validation accuracy: 0.9778
27 Validation accuracy: 0.9772
28 Validation accuracy: 0.9768
29 Validation accuracy: 0.9774
30 Validation accura

# Caching the Frozen Layers
Since the frozen layers won't change then during training, instead of building batches of training instances, it would give a huge boost to training to build batches of outputs from hidden layerr 2 and feed them to the training operation:

In [61]:
import numpy as np

tf.reset_default_graph()

n_inputs = 28 * 28  # MNIST
n_hidden1 = 300 # reused
n_hidden2 = 50  # reused
n_hidden3 = 50  # reused
n_hidden4 = 20  # new!
n_outputs = 10  # new!

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, name="hidden1") # reused frozen
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu, name="hidden2") # reused frozen & cached
    hidden2_stop = tf.stop_gradient(hidden2)
    hidden3 = tf.layers.dense(hidden2_stop, n_hidden3, activation=tf.nn.relu, name="hidden3") # reused, not frozen
    hidden4 = tf.layers.dense(hidden3, n_hidden4, activation=tf.nn.relu, name="hidden4") # new!
    logits = tf.layers.dense(hidden4, n_outputs, name="outputs") # new!

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

reuse_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope="hidden[123]") # regular expression
restore_saver = tf.train.Saver(reuse_vars) # to restore layers 1-3

init = tf.global_variables_initializer()
saver = tf.train.Saver()

n_batches = len(X_train) // batch_size

with tf.Session() as sess:
    init.run()
    restore_saver.restore(sess, "./my_model_final.ckpt")
    
    h2_cache = sess.run(hidden2, feed_dict={X: X_train})
    h2_cache_valid = sess.run(hidden2, feed_dict={X: X_valid})
    
    for epoch in range(n_epochs):
        shuffled_idx = np.random.permutation(mnist.train.num_examples)
        hidden2_batches = np.array_split(h2_cache[shuffled_idx], n_batches)
        y_batches = np.array_split(mnist.train.labels[shuffled_idx], n_batches)
        for hidden2_batch, y_batch in zip(hidden2_batches, y_batches):
            sess.run(training_op, feed_dict={hidden2: hidden2_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={hidden2: h2_cache_valid, y: y_valid})             # not shown
        print(epoch, "Validation accuracy:", accuracy_val) 
    
    save_path = saver.save(sess, "./my_new_model_final.ckpt")

INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt
0 Validation accuracy: 0.9632
1 Validation accuracy: 0.9702
2 Validation accuracy: 0.972
3 Validation accuracy: 0.9734
4 Validation accuracy: 0.9748
5 Validation accuracy: 0.9756
6 Validation accuracy: 0.9754
7 Validation accuracy: 0.9766
8 Validation accuracy: 0.9764
9 Validation accuracy: 0.9774
10 Validation accuracy: 0.978
11 Validation accuracy: 0.9774
12 Validation accuracy: 0.9772
13 Validation accuracy: 0.9776
14 Validation accuracy: 0.9776
15 Validation accuracy: 0.9782
16 Validation accuracy: 0.9772
17 Validation accuracy: 0.978
18 Validation accuracy: 0.9778
19 Validation accuracy: 0.9788
20 Validation accuracy: 0.9784
21 Validation accuracy: 0.9772
22 Validation accuracy: 0.979
23 Validation accuracy: 0.9788
24 Validation accuracy: 0.9784
25 Validation accuracy: 0.9782
26 Validation accuracy: 0.9782
27 Validation accuracy: 0.9782
28 Validation accuracy: 0.9782
29 Validation accuracy: 0.9788
30 Validation accura

# Learning Rate Scheduling
If you start with a high learning rate and then reduce it once it stops making fast progress, you can reach a good solution faster than with the optimal constant learning rate. There are many different strategies to reduce the learning rate during training. These strategies are called learning schedules. We are implementing exponential scheduling here:

In [85]:
tf.reset_default_graph()

n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 50
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=None, name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu, name="hidden2")
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")
    
with tf.name_scope("train"):
    # The learning rate will drop by a factor of 10 (decay_rate) every 10000 (decay_steps) steps.
    initial_learning_rate = 0.1
    decay_steps = 10000
    decay_rate = 1/10
    global_step = tf.Variable(0, trainable=False, name="global_step")
    learning_rate = tf.train.exponential_decay(initial_learning_rate, global_step, decay_steps, decay_rate)
    optimizer = tf.train.AdamOptimizer(learning_rate)
    training_op = optimizer.minimize(loss, global_step=global_step)
# After setting the hyperparameter values, we create a nontrainable variable global_step (initialized to 0) to keep
# track of the current training iteration number.

init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [86]:
n_epochs = 20
batch_size = 64

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Validation accuracy:", accuracy_val)
        
    save_path = saver.save(sess, "./my_model_final.ckpt")

0 Validation accuracy: 0.7524
1 Validation accuracy: 0.7822
2 Validation accuracy: 0.8312
3 Validation accuracy: 0.8656
4 Validation accuracy: 0.8738
5 Validation accuracy: 0.8908
6 Validation accuracy: 0.8842
7 Validation accuracy: 0.8934
8 Validation accuracy: 0.8968
9 Validation accuracy: 0.9042
10 Validation accuracy: 0.9062
11 Validation accuracy: 0.9114
12 Validation accuracy: 0.9064
13 Validation accuracy: 0.9144
14 Validation accuracy: 0.9152
15 Validation accuracy: 0.9148
16 Validation accuracy: 0.9162
17 Validation accuracy: 0.9188
18 Validation accuracy: 0.9206
19 Validation accuracy: 0.921


#  L1 and L2 regularization
We can also use l1 and l2 regularization to constrain a neural network's connection weights (but typically not its biases).

In [87]:
# For example, assuming you have just one hidden layer with weights W1 and one output layer with weights W2, then 
# you can apply L1 regularization like this:

tf.reset_default_graph()

n_inputs = 28 * 28  
n_hidden1 = 300
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, name="hidden1")
    logits = tf.layers.dense(hidden1, n_outputs, name="outputs")

In [89]:
W1 = tf.get_default_graph().get_tensor_by_name("hidden1/kernel:0")
W2 = tf.get_default_graph().get_tensor_by_name("outputs/kernel:0")

scale = 0.001 # L1 regularization parameter

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    base_loss = tf.reduce_mean(xentropy, name="avg_xentropy")
    reg_losses = tf.reduce_sum(tf.abs(W1)) + tf.reduce_sum(tf.abs(W2))
    loss = tf.add(base_loss, scale * reg_losses, name="loss")

In [90]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

n_epochs = 20
batch_size = 64

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Validation accuracy:", accuracy_val)

    save_path = saver.save(sess, "./my_model_final.ckpt")

0 Validation accuracy: 0.826
1 Validation accuracy: 0.865
2 Validation accuracy: 0.8802
3 Validation accuracy: 0.8906
4 Validation accuracy: 0.8954
5 Validation accuracy: 0.8998
6 Validation accuracy: 0.9028
7 Validation accuracy: 0.9046
8 Validation accuracy: 0.9068
9 Validation accuracy: 0.9074
10 Validation accuracy: 0.9076
11 Validation accuracy: 0.908
12 Validation accuracy: 0.9082
13 Validation accuracy: 0.908
14 Validation accuracy: 0.9094
15 Validation accuracy: 0.9094
16 Validation accuracy: 0.9082
17 Validation accuracy: 0.9076
18 Validation accuracy: 0.9072
19 Validation accuracy: 0.9068


TensorFlow provides a better option when there are many layers. Many function that create variables (such as **get_variable()** or **tf.layers.dense()**) accept a ***_regularizer** argument for each created variable.You can pass any function that takes weights as an argument and returns the corresponding regularization loss. 

In [101]:
tf.reset_default_graph()

n_inputs = 28 * 28  # MNIST
n_hidden1 = 300
n_hidden2 = 50
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

This code creates a neural network with two hidden layers and one output layer, and it also creates nodes in the graph to compute the L1 regularization loss corresponding to each layer's weights. TensorFlwo automatically add these nodes to a special collection containing all the regularization losses.

In [102]:
scale = 0.001

my_dense_layer = partial(tf.layers.dense, activation=tf.nn.relu, 
                         kernel_regularizer=tf.contrib.layers.l1_regularizer(scale))

with tf.name_scope("dnn"):
    hidden1 = my_dense_layer(X, n_hidden1, name="hidden1")
    hidden2 = my_dense_layer(hidden1, n_hidden2, name="hidden2")
    logits = my_dense_layer(hidden2, n_outputs, activation=None, name="outputs")

In [103]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    base_loss = tf.reduce_mean(xentropy, name="avg_xentropy")
    reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    loss = tf.add_n([base_loss] + reg_losses, name="loss")

In [104]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

n_epochs = 20
batch_size = 64

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Validation accuracy:", accuracy_val)

    save_path = saver.save(sess, "./my_model_final.ckpt")

0 Validation accuracy: 0.8846
1 Validation accuracy: 0.9094
2 Validation accuracy: 0.9156
3 Validation accuracy: 0.919
4 Validation accuracy: 0.917
5 Validation accuracy: 0.919
6 Validation accuracy: 0.917
7 Validation accuracy: 0.916
8 Validation accuracy: 0.9116
9 Validation accuracy: 0.9136
10 Validation accuracy: 0.9156
11 Validation accuracy: 0.9156
12 Validation accuracy: 0.9162
13 Validation accuracy: 0.9176
14 Validation accuracy: 0.9216
15 Validation accuracy: 0.9208
16 Validation accuracy: 0.9238
17 Validation accuracy: 0.9248
18 Validation accuracy: 0.9294
19 Validation accuracy: 0.9296


# Dropout
To implement dropout using TensorFlow, you can simply apply the **tf.layers.dropout()** function to the input layer and/or to the output of any hidden layer you want. During training, this function randomly drops some items (setting them to 0) and divides the remaining items by the keep probability. After training, this function does nothing at all. The following code applies dropout regularization to our three-layer neural network:

In [108]:
tf.reset_default_graph()

n_inputs = 28 * 28  # MNIST
n_hidden1 = 300
n_hidden2 = 50
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

In [109]:
training = tf.placeholder_with_default(False, shape=(), name="training")

dropout_rate = 0.5 
X_drop = tf.layers.dropout(X, dropout_rate, training=training)

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, name="hidden1")
    hidden1_drop = tf.layers.dropout(hidden1, dropout_rate, training=training)
    hidden2 = tf.layers.dense(hidden1_drop, n_hidden2, activation=tf.nn.relu, name="hidden2")
    hidden2_drop = tf.layers.dropout(hidden2, dropout_rate, training=training)
    logits = tf.layers.dense(hidden2_drop, n_outputs, name="outputs")

In [110]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("train"):
    optimizer = tf.train.MomentumOptimizer(learning_rate, momentum=0.9)
    training_op = optimizer.minimize(loss)    

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [111]:
# We need to set training to True only when training, and leave the default False value when testing
n_epochs = 20
batch_size = 64

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch, training: True})
        accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Validation accuracy:", accuracy_val)

    save_path = saver.save(sess, "./my_model_final.ckpt")

0 Validation accuracy: 0.9418
1 Validation accuracy: 0.9544
2 Validation accuracy: 0.964
3 Validation accuracy: 0.966
4 Validation accuracy: 0.9698
5 Validation accuracy: 0.9724
6 Validation accuracy: 0.975
7 Validation accuracy: 0.975
8 Validation accuracy: 0.9768
9 Validation accuracy: 0.9764
10 Validation accuracy: 0.9766
11 Validation accuracy: 0.978
12 Validation accuracy: 0.9772
13 Validation accuracy: 0.9788
14 Validation accuracy: 0.9798
15 Validation accuracy: 0.979
16 Validation accuracy: 0.981
17 Validation accuracy: 0.9794
18 Validation accuracy: 0.98
19 Validation accuracy: 0.9794


**Note1:** If you observe that the model is overfitting, you can increase the dropout rate. Conversely, you should try decreasing the dropout rate if the model underfits the training set. <br>
**Note2:** Dropconnect is a variant of dropout where individual conenctions are dropped randomly rather than whole neurons. In general dropout performs better.

# Max-Norm Regularization
- Another regularization technique that is quite popular for neural networks is called max-norm regularization: for each neuron, it constrains the weights **w** of the incoming connections such that ||**w**||$_{2}$ $\leq$ r, where r is the max-norm hyperparameter and ||.||$_{2}$ is the l2 norm.<br>
- Reducing r increases the amount of regularization and helps reduce overfitting. Max-norm regularization can also help alleviate the vanishing/exploding gradients problems (if you are not using Batch Normalization). <br>



In [118]:
tf.reset_default_graph()

n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 50
n_outputs = 10

learning_rate = 0.01
momentum = 0.9

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu, name="hidden2")
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("train"):
    optimizer = tf.train.MomentumOptimizer(learning_rate, momentum)
    training_op = optimizer.minimize(loss)    

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

TensorFlow does not provide an off-the-shelf max-norm regularizer, but it is not too hard to implement. The following code gets a handle on the weights of the first and second hidden layer, then it uses the **clip_by_norm()** function to create an operation that will clip the weights along the second axis so that each row vector ends up with a maximum norm of 1.0. The last line creates an assignment operation that will assign the clipped weights to the weights variable:

In [119]:
threshold = 1.0
weights = tf.get_default_graph().get_tensor_by_name("hidden1/kernel:0")
clipped_weights = tf.clip_by_norm(weights, clip_norm=threshold, axes=1)
clip_weights = tf.assign(weights, clipped_weights)

In [120]:
weights2 = tf.get_default_graph().get_tensor_by_name("hidden2/kernel:0")
clipped_weights2 = tf.clip_by_norm(weights, clip_norm=threshold, axes=1)
clip_weights2 = tf.assign(weights, clipped_weights)

In [121]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [122]:
n_epochs = 20
batch_size = 64

with tf.Session() as sess:                                              
    init.run()                                                          
    for epoch in range(n_epochs):                                       
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size): 
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            clip_weights.eval()
            clip_weights2.eval()                                        
        acc_valid = accuracy.eval(feed_dict={X: X_valid, y: y_valid})   
        print(epoch, "Validation accuracy:", acc_valid)                 

    save_path = saver.save(sess, "./my_model_final.ckpt")   

0 Validation accuracy: 0.9538
1 Validation accuracy: 0.9654
2 Validation accuracy: 0.9696
3 Validation accuracy: 0.975
4 Validation accuracy: 0.9766
5 Validation accuracy: 0.9774
6 Validation accuracy: 0.9788
7 Validation accuracy: 0.977
8 Validation accuracy: 0.9774
9 Validation accuracy: 0.9772
10 Validation accuracy: 0.9782
11 Validation accuracy: 0.9792
12 Validation accuracy: 0.9806
13 Validation accuracy: 0.9812
14 Validation accuracy: 0.9836
15 Validation accuracy: 0.9802
16 Validation accuracy: 0.9806
17 Validation accuracy: 0.9814
18 Validation accuracy: 0.9812
19 Validation accuracy: 0.9816


When we want to do this for every hidden layer, we can create a **max_norm_regularizer()** function and use it just like the earlier **l1_regularizer()** function:

In [129]:
tf.reset_default_graph()

n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 50
n_outputs = 10

learning_rate = 0.01
momentum = 0.9

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

In [130]:
def max_norm_regularizer(threshold, axes=1, name="max_norm", collection="max_norm"):
    def max_norm(weights):
        clipped = tf.clip_by_norm(weights, clip_norm=threshold, axes=axes)
        clip_weights = tf.assign(weights, clipped, name=name)
        tf.add_to_collection(collection, clip_weights)
        return None # there is no regularization loss term
    return max_norm

This function returns a parametrized **max_norm()** function that you can use like any other regularizer:

In [131]:
max_norm_reg = max_norm_regularizer(threshold=1.0)

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_inputs, activation=tf.nn.relu, kernel_regularizer=max_norm_reg, name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu, kernel_regularizer=max_norm_reg, name="hidden2")
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")

In [132]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("train"):
    optimizer = tf.train.MomentumOptimizer(learning_rate, momentum)
    training_op = optimizer.minimize(loss)    

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [134]:
n_epochs = 20
batch_size = 64

In [135]:
clip_all_weights = tf.get_collection("max_norm")

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            sess.run(clip_all_weights)
        accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Validation accuracy:", accuracy_val)
    
    save_path = saver.save(sess, "./my_model_final.ckpt")

0 Validation accuracy: 0.952
1 Validation accuracy: 0.9688
2 Validation accuracy: 0.9752
3 Validation accuracy: 0.9738
4 Validation accuracy: 0.9814
5 Validation accuracy: 0.9806
6 Validation accuracy: 0.9826
7 Validation accuracy: 0.9816
8 Validation accuracy: 0.9836
9 Validation accuracy: 0.9846
10 Validation accuracy: 0.9848
11 Validation accuracy: 0.982
12 Validation accuracy: 0.9844
13 Validation accuracy: 0.9842
14 Validation accuracy: 0.9838
15 Validation accuracy: 0.9848
16 Validation accuracy: 0.984
17 Validation accuracy: 0.9842
18 Validation accuracy: 0.9858
19 Validation accuracy: 0.9854


**Exercise:** Build a DNN with five hidden layers of 100 neurons each, He initialization, and the ELU activation function.

In [17]:
tf.reset_default_graph()

n_inputs = 28 * 28
n_hidden1 = 100
n_hidden2 = 100
n_hidden3 = 100
n_hidden4 = 100
n_hidden5 = 100
n_outputs = 5

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=None, name="y")

he_init = tf.variance_scaling_initializer()

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.elu, kernel_initializer=he_init, name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.elu, kernel_initializer=he_init, name="hidden2")
    hidden3 = tf.layers.dense(hidden2, n_hidden3, activation=tf.nn.elu, kernel_initializer=he_init, name="hidden3")
    hidden4 = tf.layers.dense(hidden3, n_hidden4, activation=tf.nn.elu, kernel_initializer=he_init, name="hidden4")
    hidden5 = tf.layers.dense(hidden4, n_hidden5, activation=tf.nn.elu, kernel_initializer=he_init, name="hidden5")

logits = tf.layers.dense(hidden5, n_outputs, kernel_initializer=he_init, name="outputs")
Y_proba = tf.nn.softmax(logits, name="Y_proba")

**Exercise:** Using Adam optimization and early stopping, try training it on MNIST but only on digits 0 to 4, as we will use transfer learning for digits 5 to 9 in the next exercise. You will need a softmax output layer with five neurons, and as always make sure to save checkpoints at regular intervals and save the final model so you can reuse it later.

In [18]:
learning_rate = 0.01

xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
loss = tf.reduce_mean(xentropy, name="loss")

optimizer = tf.train.AdamOptimizer(learning_rate)
training_op = optimizer.minimize(loss, name="training_op")

correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

init = tf.global_variables_initializer()
saver = tf.train.Saver()

Now let's create the training set, validation and test set (we need the validation set to implement early stopping):

In [19]:
X_train1 = X_train[y_train < 5]
y_train1 = y_train[y_train < 5]
X_valid1 = X_valid[y_valid < 5]
y_valid1 = y_valid[y_valid < 5]
X_test1 = X_test[y_test < 5]
y_test1 = y_test[y_test < 5]

In [20]:
n_epochs = 1000
batch_size = 32

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

with tf.Session() as sess:
    init.run()
    
    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(len(X_train1))
        for rnd_indices in np.array_split(rnd_idx, len(X_train1) // batch_size):
            X_batch, y_batch = X_train1[rnd_indices], y_train1[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={X: X_valid1, y: y_valid1})
        if loss_val < best_loss:
            save_path = saver.save(sess, "./my_mnist_model_0_to_4.ckpt")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\t Accuracy: {:.2f}%".format(epoch, loss_val, best_loss, acc_val * 100))

with tf.Session() as sess:
    saver.restore(sess, "./my_mnist_model_0_to_4.ckpt")
    acc_test = accuracy.eval(feed_dict={X: X_test1, y: y_test1})
    print("Final test accuracy: {:.2f}%".format(acc_test * 100))

0	Validation loss: 0.158282	Best loss: 0.158282	 Accuracy: 95.58%
1	Validation loss: 0.082704	Best loss: 0.082704	 Accuracy: 98.28%
2	Validation loss: 0.123639	Best loss: 0.082704	 Accuracy: 96.72%
3	Validation loss: 0.069824	Best loss: 0.069824	 Accuracy: 98.48%
4	Validation loss: 0.250361	Best loss: 0.069824	 Accuracy: 97.07%
5	Validation loss: 0.065760	Best loss: 0.065760	 Accuracy: 98.75%
6	Validation loss: 0.051657	Best loss: 0.051657	 Accuracy: 98.71%
7	Validation loss: 0.107035	Best loss: 0.051657	 Accuracy: 97.81%
8	Validation loss: 0.103435	Best loss: 0.051657	 Accuracy: 97.42%
9	Validation loss: 0.101946	Best loss: 0.051657	 Accuracy: 97.62%
10	Validation loss: 0.391774	Best loss: 0.051657	 Accuracy: 96.21%
11	Validation loss: 1.206018	Best loss: 0.051657	 Accuracy: 95.82%
12	Validation loss: 0.270881	Best loss: 0.051657	 Accuracy: 97.15%
13	Validation loss: 0.354061	Best loss: 0.051657	 Accuracy: 93.28%
14	Validation loss: 0.151958	Best loss: 0.051657	 Accuracy: 97.89%
15	Va

**Exercise:** Tune the hyperparameters using cross-validation and see what precision you can achieve.

Let's create a DNNClassifier class, compatible with Scikit-Learn's RandomizedSearchCV class, to perform hyperparameter tuning. Here are the key points of this implementation:
- the **\_\_init\_\_()** method (constructor) does nothing more than create instance variables for each of the hyperparameters.
- the **fit()** method creates the graph, starts a session and trains the model:
    - it calls the _build_graph() method to build the graph (much like the graph we defined earlier). Once this method is done creating the graph, it saves all the important operations as instance variables for easy access by other methods.
    - the _dnn() method builds the hidden layers, just like the dnn() function above, but also with support for batch normalization and dropout (for the next exercises).
    - if the fit() method is given a validation set (X_valid and y_valid), then it implements early stopping. This implementation does not save the best model to disk, but rather to memory: it uses the _get_model_params() method to get all the graph's variables and their values, and the _restore_model_params() method to restore the variable values (of the best model found). This trick helps speed up training.
    -  After the fit() method has finished training the model, it keeps the session open so that predictions can be made quickly, without having to save a model to disk and restore it for every prediction. You can close the session by calling the close_session() method.
- the **predict_proba()** method uses the trained model to predict the class probabilities.
- the **predict()** method calls predict_proba() and returns the class with the highest probability, for each instance.

In [21]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.exceptions import NotFittedError

class DNNClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, n_hidden_layers=5, n_neurons=100, optimizer_class=tf.train.AdamOptimizer,
                 learning_rate=0.01, batch_size=32, activation=tf.nn.elu, initializer=he_init,
                 batch_norm_momentum=None, dropout_rate=None, random_state=None):
        """Initialize the DNNClassifier by simply storing all the hyperparameters."""
        self.n_hidden_layers = n_hidden_layers
        self.n_neurons = n_neurons
        self.optimizer_class = optimizer_class
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.activation = activation
        self.initializer = initializer
        self.batch_norm_momentum = batch_norm_momentum
        self.dropout_rate = dropout_rate
        self.random_state = random_state
        self._session = None

    def _dnn(self, inputs):
        """Build the hidden layers, with support for batch normalization and dropout."""
        for layer in range(self.n_hidden_layers):
            if self.dropout_rate:
                inputs = tf.layers.dropout(inputs, self.dropout_rate, training=self._training)
            inputs = tf.layers.dense(inputs, self.n_neurons,
                                     kernel_initializer=self.initializer,
                                     name="hidden%d" % (layer + 1))
            if self.batch_norm_momentum:
                inputs = tf.layers.batch_normalization(inputs, momentum=self.batch_norm_momentum,
                                                       training=self._training)
            inputs = self.activation(inputs, name="hidden%d_out" % (layer + 1))
        return inputs

    def _build_graph(self, n_inputs, n_outputs):
        """Build the same model as earlier"""
        if self.random_state is not None:
            tf.set_random_seed(self.random_state)
            np.random.seed(self.random_state)

        X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
        y = tf.placeholder(tf.int32, shape=(None), name="y")

        if self.batch_norm_momentum or self.dropout_rate:
            self._training = tf.placeholder_with_default(False, shape=(), name='training')
        else:
            self._training = None

        dnn_outputs = self._dnn(X)

        logits = tf.layers.dense(dnn_outputs, n_outputs, kernel_initializer=he_init, name="logits")
        Y_proba = tf.nn.softmax(logits, name="Y_proba")

        xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                                  logits=logits)
        loss = tf.reduce_mean(xentropy, name="loss")

        optimizer = self.optimizer_class(learning_rate=self.learning_rate)
        training_op = optimizer.minimize(loss)

        correct = tf.nn.in_top_k(logits, y, 1)
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

        init = tf.global_variables_initializer()
        saver = tf.train.Saver()

        # Make the important operations available easily through instance variables
        self._X, self._y = X, y
        self._Y_proba, self._loss = Y_proba, loss
        self._training_op, self._accuracy = training_op, accuracy
        self._init, self._saver = init, saver

    def close_session(self):
        if self._session:
            self._session.close()

    def _get_model_params(self):
        """Get all variable values (used for early stopping, faster than saving to disk)"""
        with self._graph.as_default():
            gvars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
        return {gvar.op.name: value for gvar, value in zip(gvars, self._session.run(gvars))}

    def _restore_model_params(self, model_params):
        """Set all variables to the given values (for early stopping, faster than loading from disk)"""
        gvar_names = list(model_params.keys())
        assign_ops = {gvar_name: self._graph.get_operation_by_name(gvar_name + "/Assign")
                      for gvar_name in gvar_names}
        init_values = {gvar_name: assign_op.inputs[1] for gvar_name, assign_op in assign_ops.items()}
        feed_dict = {init_values[gvar_name]: model_params[gvar_name] for gvar_name in gvar_names}
        self._session.run(assign_ops, feed_dict=feed_dict)

    def fit(self, X, y, n_epochs=100, X_valid=None, y_valid=None):
        """Fit the model to the training set. If X_valid and y_valid are provided, use early stopping."""
        self.close_session()

        # infer n_inputs and n_outputs from the training set.
        n_inputs = X.shape[1]
        self.classes_ = np.unique(y)
        n_outputs = len(self.classes_)
        
        # Translate the labels vector to a vector of sorted class indices, containing
        # integers from 0 to n_outputs - 1.
        # For example, if y is equal to [8, 8, 9, 5, 7, 6, 6, 6], then the sorted class
        # labels (self.classes_) will be equal to [5, 6, 7, 8, 9], and the labels vector
        # will be translated to [3, 3, 4, 0, 2, 1, 1, 1]
        self.class_to_index_ = {label: index
                                for index, label in enumerate(self.classes_)}
        y = np.array([self.class_to_index_[label]
                      for label in y], dtype=np.int32)
        
        self._graph = tf.Graph()
        with self._graph.as_default():
            self._build_graph(n_inputs, n_outputs)
            # extra ops for batch normalization
            extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

        # needed in case of early stopping
        max_checks_without_progress = 20
        checks_without_progress = 0
        best_loss = np.infty
        best_params = None
        
        # Now train the model!
        self._session = tf.Session(graph=self._graph)
        with self._session.as_default() as sess:
            self._init.run()
            for epoch in range(n_epochs):
                rnd_idx = np.random.permutation(len(X))
                for rnd_indices in np.array_split(rnd_idx, len(X) // self.batch_size):
                    X_batch, y_batch = X[rnd_indices], y[rnd_indices]
                    feed_dict = {self._X: X_batch, self._y: y_batch}
                    if self._training is not None:
                        feed_dict[self._training] = True
                    sess.run(self._training_op, feed_dict=feed_dict)
                    if extra_update_ops:
                        sess.run(extra_update_ops, feed_dict=feed_dict)
                if X_valid is not None and y_valid is not None:
                    loss_val, acc_val = sess.run([self._loss, self._accuracy],
                                                 feed_dict={self._X: X_valid,
                                                            self._y: y_valid})
                    if loss_val < best_loss:
                        best_params = self._get_model_params()
                        best_loss = loss_val
                        checks_without_progress = 0
                    else:
                        checks_without_progress += 1
                    print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
                        epoch, loss_val, best_loss, acc_val * 100))
                    if checks_without_progress > max_checks_without_progress:
                        print("Early stopping!")
                        break
                else:
                    loss_train, acc_train = sess.run([self._loss, self._accuracy],
                                                     feed_dict={self._X: X_batch,
                                                                self._y: y_batch})
                    print("{}\tLast training batch loss: {:.6f}\tAccuracy: {:.2f}%".format(
                        epoch, loss_train, acc_train * 100))
            # If we used early stopping then rollback to the best model found
            if best_params:
                self._restore_model_params(best_params)
            return self

    def predict_proba(self, X):
        if not self._session:
            raise NotFittedError("This %s instance is not fitted yet" % self.__class__.__name__)
        with self._session.as_default() as sess:
            return self._Y_proba.eval(feed_dict={self._X: X})

    def predict(self, X):
        class_indices = np.argmax(self.predict_proba(X), axis=1)
        return np.array([[self.classes_[class_index]]
                         for class_index in class_indices], np.int32)

    def save(self, path):
        self._saver.save(self._session, path)

In [22]:
tf.reset_default_graph()

dnn_clf = DNNClassifier(random_state=42)
dnn_clf.fit(X_train1, y_train1, n_epochs=1000, X_valid=X_valid1, y_valid=y_valid1)

0	Validation loss: 0.115825	Best loss: 0.115825	Accuracy: 97.19%
1	Validation loss: 0.177427	Best loss: 0.115825	Accuracy: 96.01%
2	Validation loss: 0.107341	Best loss: 0.107341	Accuracy: 97.93%
3	Validation loss: 0.073096	Best loss: 0.073096	Accuracy: 98.48%
4	Validation loss: 0.151962	Best loss: 0.073096	Accuracy: 96.52%
5	Validation loss: 0.182587	Best loss: 0.073096	Accuracy: 96.99%
6	Validation loss: 0.129222	Best loss: 0.073096	Accuracy: 96.87%
7	Validation loss: 0.090236	Best loss: 0.073096	Accuracy: 97.73%
8	Validation loss: 0.132752	Best loss: 0.073096	Accuracy: 97.62%
9	Validation loss: 0.106750	Best loss: 0.073096	Accuracy: 98.12%
10	Validation loss: 0.353080	Best loss: 0.073096	Accuracy: 90.70%
11	Validation loss: 0.100497	Best loss: 0.073096	Accuracy: 98.05%
12	Validation loss: 0.140937	Best loss: 0.073096	Accuracy: 97.50%
13	Validation loss: 0.086660	Best loss: 0.073096	Accuracy: 98.24%
14	Validation loss: 0.154946	Best loss: 0.073096	Accuracy: 97.62%
15	Validation loss: 

DNNClassifier(activation=<function elu at 0x7fade4683dd0>,
              batch_norm_momentum=None, batch_size=32, dropout_rate=None,
              initializer=<tensorflow.python.ops.init_ops.VarianceScaling object at 0x7fad6f31d8d0>,
              learning_rate=0.01, n_hidden_layers=5, n_neurons=100,
              optimizer_class=<class 'tensorflow.python.training.adam.AdamOptimizer'>,
              random_state=42)

The model is trained, let's see if it gets the same accuracy as earlier:

In [23]:
from sklearn.metrics import accuracy_score

y_pred = dnn_clf.predict(X_test1)
accuracy_score(y_test1, y_pred)

0.9873516248297334

Yep! Working fine. Now we can use Scikit-Learn's RandomizedSearchCV class to search for better hyperparameters (this may take over an hour, depending on your system):

In [24]:
from sklearn.model_selection import RandomizedSearchCV

def leaky_relu(alpha=0.01):
    def parametrized_leaky_relu(z, name=None):
        return tf.maximum(alpha * z, z, name=name)
    return parametrized_leaky_relu

param_distribs = {
    "n_neurons": [10, 30, 50, 70, 90, 100, 120, 140, 160],
    "batch_size": [16, 64, 128, 512],
    "learning_rate": [0.01, 0.02, 0.05, 0.1],
    "activation": [tf.nn.relu, tf.nn.elu, leaky_relu(alpha=0.01), leaky_relu(alpha=0.1)],
    # you could also try exploring different numbers of hidden layers, different optimizers, etc.
    #"n_hidden_layers": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    #"optimizer_class": [tf.train.AdamOptimizer, partial(tf.train.MomentumOptimizer, momentum=0.95)],
}

rnd_search = RandomizedSearchCV(DNNClassifier(random_state=42), param_distribs, n_iter=50,
                                cv=3, random_state=42, verbose=2)
rnd_search.fit(X_train1, y_train1, X_valid=X_valid1, y_valid=y_valid1, n_epochs=1000)

Fitting 3 folds for each of 50 candidates, totalling 150 fits
[CV] n_neurons=10, learning_rate=0.05, batch_size=128, activation=<function elu at 0x7fade4683dd0> 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0	Validation loss: 0.145495	Best loss: 0.145495	Accuracy: 95.86%
1	Validation loss: 0.147682	Best loss: 0.145495	Accuracy: 96.25%
2	Validation loss: 0.112825	Best loss: 0.112825	Accuracy: 96.91%
3	Validation loss: 0.110635	Best loss: 0.110635	Accuracy: 97.38%
4	Validation loss: 0.168859	Best loss: 0.110635	Accuracy: 95.74%
5	Validation loss: 0.122974	Best loss: 0.110635	Accuracy: 97.03%
6	Validation loss: 0.129440	Best loss: 0.110635	Accuracy: 96.36%
7	Validation loss: 0.129536	Best loss: 0.110635	Accuracy: 96.64%
8	Validation loss: 3.125185	Best loss: 0.110635	Accuracy: 31.00%
9	Validation loss: 1.299110	Best loss: 0.110635	Accuracy: 37.61%
10	Validation loss: 1.209157	Best loss: 0.110635	Accuracy: 39.60%
11	Validation loss: 1.156154	Best loss: 0.110635	Accuracy: 40.03%
12	Validation loss: 1.161423	Best loss: 0.110635	Accuracy: 39.60%
13	Validation loss: 1.164427	Best loss: 0.110635	Accuracy: 42.06%
14	Validation loss: 1.168357	Best loss: 0.110635	Accuracy: 39.68%
15	Validation loss: 

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    7.2s remaining:    0.0s


0	Validation loss: 0.130183	Best loss: 0.130183	Accuracy: 96.21%
1	Validation loss: 0.123591	Best loss: 0.123591	Accuracy: 96.17%
2	Validation loss: 0.149474	Best loss: 0.123591	Accuracy: 96.40%
3	Validation loss: 0.158407	Best loss: 0.123591	Accuracy: 96.68%
4	Validation loss: 0.151179	Best loss: 0.123591	Accuracy: 97.65%
5	Validation loss: 0.137467	Best loss: 0.123591	Accuracy: 96.91%
6	Validation loss: 0.151529	Best loss: 0.123591	Accuracy: 96.87%
7	Validation loss: 0.121131	Best loss: 0.121131	Accuracy: 97.30%
8	Validation loss: 0.117541	Best loss: 0.117541	Accuracy: 97.38%
9	Validation loss: 0.124826	Best loss: 0.117541	Accuracy: 97.15%
10	Validation loss: 0.165149	Best loss: 0.117541	Accuracy: 97.30%
11	Validation loss: 0.181092	Best loss: 0.117541	Accuracy: 95.47%
12	Validation loss: 1.645416	Best loss: 0.117541	Accuracy: 20.91%
13	Validation loss: 1.631544	Best loss: 0.117541	Accuracy: 19.08%
14	Validation loss: 1.623457	Best loss: 0.117541	Accuracy: 22.01%
15	Validation loss: 

...

27	Validation loss: 0.515089	Best loss: 0.427560	Accuracy: 97.26%
28	Validation loss: 0.495225	Best loss: 0.427560	Accuracy: 96.91%
29	Validation loss: 0.436487	Best loss: 0.427560	Accuracy: 96.91%
30	Validation loss: 0.386677	Best loss: 0.386677	Accuracy: 97.42%
31	Validation loss: 0.398355	Best loss: 0.386677	Accuracy: 96.83%
32	Validation loss: 0.504766	Best loss: 0.386677	Accuracy: 96.87%
33	Validation loss: 0.373899	Best loss: 0.373899	Accuracy: 97.07%
34	Validation loss: 0.399982	Best loss: 0.373899	Accuracy: 96.91%
35	Validation loss: 0.492959	Best loss: 0.373899	Accuracy: 97.11%
36	Validation loss: 0.336339	Best loss: 0.336339	Accuracy: 97.50%
37	Validation loss: 0.380107	Best loss: 0.336339	Accuracy: 97.58%
38	Validation loss: 0.787642	Best loss: 0.336339	Accuracy: 95.86%
39	Validation loss: 0.521611	Best loss: 0.336339	Accuracy: 96.95%
40	Validation loss: 0.591301	Best loss: 0.336339	Accuracy: 96.99%
41	Validation loss: 0.326864	Best loss: 0.326864	Accuracy: 97.62%
42	Validat

0	Validation loss: 944.805115	Best loss: 944.805115	Accuracy: 51.17%
1	Validation loss: 21.952637	Best loss: 21.952637	Accuracy: 87.88%
2	Validation loss: 3.643055	Best loss: 3.643055	Accuracy: 92.65%
3	Validation loss: 2.566448	Best loss: 2.566448	Accuracy: 92.69%
4	Validation loss: 1.170726	Best loss: 1.170726	Accuracy: 95.27%
5	Validation loss: 1.491794	Best loss: 1.170726	Accuracy: 92.46%
6	Validation loss: 1.406537	Best loss: 1.170726	Accuracy: 95.39%
7	Validation loss: 1.860583	Best loss: 1.170726	Accuracy: 91.28%
8	Validation loss: 1.062331	Best loss: 1.062331	Accuracy: 93.47%
9	Validation loss: 0.629030	Best loss: 0.629030	Accuracy: 95.15%
10	Validation loss: 1.679119	Best loss: 0.629030	Accuracy: 95.39%
11	Validation loss: 0.820489	Best loss: 0.629030	Accuracy: 96.79%
12	Validation loss: 0.621473	Best loss: 0.621473	Accuracy: 95.93%
13	Validation loss: 1.433000	Best loss: 0.621473	Accuracy: 87.06%
14	Validation loss: 1.482834	Best loss: 0.621473	Accuracy: 96.09%
15	Validation 

[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed: 51.3min finished


0	Validation loss: 0.087587	Best loss: 0.087587	Accuracy: 97.50%
1	Validation loss: 0.054727	Best loss: 0.054727	Accuracy: 98.12%
2	Validation loss: 0.054701	Best loss: 0.054701	Accuracy: 98.40%
3	Validation loss: 0.059889	Best loss: 0.054701	Accuracy: 98.20%
4	Validation loss: 0.052853	Best loss: 0.052853	Accuracy: 98.40%
5	Validation loss: 0.050474	Best loss: 0.050474	Accuracy: 98.75%
6	Validation loss: 0.047548	Best loss: 0.047548	Accuracy: 98.83%
7	Validation loss: 0.044886	Best loss: 0.044886	Accuracy: 98.91%
8	Validation loss: 0.051252	Best loss: 0.044886	Accuracy: 98.87%
9	Validation loss: 0.046965	Best loss: 0.044886	Accuracy: 98.67%
10	Validation loss: 0.052950	Best loss: 0.044886	Accuracy: 98.51%
11	Validation loss: 0.056926	Best loss: 0.044886	Accuracy: 99.02%
12	Validation loss: 0.063791	Best loss: 0.044886	Accuracy: 98.79%
13	Validation loss: 0.050034	Best loss: 0.044886	Accuracy: 98.67%
14	Validation loss: 0.052112	Best loss: 0.044886	Accuracy: 98.75%
15	Validation loss: 

RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=DNNClassifier(activation=<function elu at 0x7fade4683dd0>,
                                           batch_norm_momentum=None,
                                           batch_size=32, dropout_rate=None,
                                           initializer=<tensorflow.python.ops.init_ops.VarianceScaling object at 0x7fad6f31d8d0>,
                                           learning_rate=0.01,
                                           n_hidden_layers=5, n_neurons=100,
                                           optimizer_class=<class 'tensorflow.python.training.adam...
                                                       <function leaky_relu.<locals>.parametrized_leaky_relu at 0x7fadbd85b9e0>,
                                                       <function leaky_relu.<locals>.parametrized_leaky_relu at 0x7fadbd85be60>],
                                        'batch_size': [16, 64, 128, 512],
                   

In [25]:
rnd_search.best_params_

{'n_neurons': 140,
 'learning_rate': 0.01,
 'batch_size': 512,
 'activation': <function __main__.leaky_relu.<locals>.parametrized_leaky_relu(z, name=None)>}

In [26]:
y_pred = rnd_search.predict(X_test1)
accuracy_score(y_test1, y_pred)

0.9900758902510216

In [27]:
# Let's save this model
rnd_search.best_estimator_.save("./my_best_mnist_model_0_to_4")

**Exercise:** Now try adding Batch Normalization and compare the learning curves: is it converging faster than before? Does it produce a better model?

In [28]:
dnn_clf = DNNClassifier(activation=leaky_relu(alpha=0.1), batch_size=500, learning_rate=0.01,
                        n_neurons=140, random_state=42)
dnn_clf.fit(X_train1, y_train1, n_epochs=1000, X_valid=X_valid1, y_valid=y_valid1)

0	Validation loss: 0.088670	Best loss: 0.088670	Accuracy: 97.30%
1	Validation loss: 0.052064	Best loss: 0.052064	Accuracy: 98.28%
2	Validation loss: 0.040267	Best loss: 0.040267	Accuracy: 98.59%
3	Validation loss: 0.115331	Best loss: 0.040267	Accuracy: 96.76%
4	Validation loss: 0.050305	Best loss: 0.040267	Accuracy: 98.55%
5	Validation loss: 0.058298	Best loss: 0.040267	Accuracy: 98.48%
6	Validation loss: 0.051559	Best loss: 0.040267	Accuracy: 98.91%
7	Validation loss: 0.053180	Best loss: 0.040267	Accuracy: 98.44%
8	Validation loss: 0.052916	Best loss: 0.040267	Accuracy: 98.75%
9	Validation loss: 0.039415	Best loss: 0.039415	Accuracy: 98.98%
10	Validation loss: 0.039494	Best loss: 0.039415	Accuracy: 98.79%
11	Validation loss: 0.056520	Best loss: 0.039415	Accuracy: 98.91%
12	Validation loss: 0.035726	Best loss: 0.035726	Accuracy: 99.02%
13	Validation loss: 0.076411	Best loss: 0.035726	Accuracy: 98.75%
14	Validation loss: 0.060660	Best loss: 0.035726	Accuracy: 98.67%
15	Validation loss: 

DNNClassifier(activation=<function leaky_relu.<locals>.parametrized_leaky_relu at 0x7fadbd4fbcb0>,
              batch_norm_momentum=None, batch_size=500, dropout_rate=None,
              initializer=<tensorflow.python.ops.init_ops.VarianceScaling object at 0x7fad6f31d8d0>,
              learning_rate=0.01, n_hidden_layers=5, n_neurons=140,
              optimizer_class=<class 'tensorflow.python.training.adam.AdamOptimizer'>,
              random_state=42)

In [29]:
y_pred = dnn_clf.predict(X_test1)
accuracy_score(y_test1, y_pred)

0.9918272037361354

In [30]:
dnn_clf_bn = DNNClassifier(activation=leaky_relu(alpha=0.1), batch_size=500, learning_rate=0.01,
                           n_neurons=90, random_state=42,
                           batch_norm_momentum=0.95)
dnn_clf_bn.fit(X_train1, y_train1, n_epochs=1000, X_valid=X_valid1, y_valid=y_valid1)

Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).


0	Validation loss: 0.045447	Best loss: 0.045447	Accuracy: 98.75%
1	Validation loss: 0.037184	Best loss: 0.037184	Accuracy: 98.75%
2	Validation loss: 0.039710	Best loss: 0.037184	Accuracy: 98.83%
3	Validation loss: 0.043848	Best loss: 0.037184	Accuracy: 98.67%
4	Validation loss: 0.053418	Best loss: 0.037184	Accuracy: 98.67%
5	Validation loss: 0.038507	Best loss: 0.037184	Accuracy: 98.75%
6	Validation loss: 0.037694	Best loss: 0.037184	Accuracy: 98.79%
7	Validation loss: 0.025464	Best loss: 0.025464	Accuracy: 99.30%
8	Validation loss: 0.031518	Best loss: 0.025464	Accuracy: 99.06%
9	Validation loss: 0.028544	Best loss: 0.025464	Accuracy: 99.06%
10	Validation loss: 0.033450	Best loss: 0.025464	Accuracy: 98.98%
11	Validation loss: 0.051673	Best loss: 0.025464	Accuracy: 98.79%
12	Validation loss: 0.038245	Best loss: 0.025464	Accuracy: 99.06%
13	Validation loss: 0.035552	Best loss: 0.025464	Accuracy: 99.10%
14	Validation loss: 0.040883	Best loss: 0.025464	Accuracy: 99.02%
15	Validation loss: 

DNNClassifier(activation=<function leaky_relu.<locals>.parametrized_leaky_relu at 0x7fad6d11e9e0>,
              batch_norm_momentum=0.95, batch_size=500, dropout_rate=None,
              initializer=<tensorflow.python.ops.init_ops.VarianceScaling object at 0x7fad6f31d8d0>,
              learning_rate=0.01, n_hidden_layers=5, n_neurons=90,
              optimizer_class=<class 'tensorflow.python.training.adam.AdamOptimizer'>,
              random_state=42)

In [31]:
y_pred = dnn_clf_bn.predict(X_test1)
accuracy_score(y_test1, y_pred)

0.9914380229616657

Wow awesome! Batch Normalization improved accuracy! To tweak hyperparameters with batch normalization, you can try RandomizedSearchCV as:

In [34]:
from sklearn.model_selection import RandomizedSearchCV

param_distribs = {
    "n_neurons": [10, 30, 50, 70, 90, 100, 120, 140, 160],
    "batch_size": [16, 64, 128, 512],
    "learning_rate": [0.01, 0.02, 0.05, 0.1],
    "activation": [tf.nn.relu, tf.nn.elu, leaky_relu(alpha=0.01), leaky_relu(alpha=0.1)],
    # you could also try exploring different numbers of hidden layers, different optimizers, etc.
    #"n_hidden_layers": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    #"optimizer_class": [tf.train.AdamOptimizer, partial(tf.train.MomentumOptimizer, momentum=0.95)],
    "batch_norm_momentum": [0.9, 0.95, 0.98, 0.99, 0.999],
}

rnd_search_bn = RandomizedSearchCV(DNNClassifier(random_state=42), param_distribs, n_iter=50, cv=3,
                                   random_state=42, verbose=2)
rnd_search_bn.fit(X_train1, y_train1, X_valid=X_valid1, y_valid=y_valid1, n_epochs=1000)


Fitting 3 folds for each of 50 candidates, totalling 150 fits
[CV] n_neurons=70, learning_rate=0.01, batch_size=64, batch_norm_momentum=0.99, activation=<function relu at 0x7fade464d9e0> 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0	Validation loss: 0.077747	Best loss: 0.077747	Accuracy: 98.20%
1	Validation loss: 0.067209	Best loss: 0.067209	Accuracy: 98.44%
2	Validation loss: 0.053466	Best loss: 0.053466	Accuracy: 98.55%
3	Validation loss: 0.043413	Best loss: 0.043413	Accuracy: 98.51%
4	Validation loss: 0.050891	Best loss: 0.043413	Accuracy: 98.36%
5	Validation loss: 0.053303	Best loss: 0.043413	Accuracy: 98.51%
6	Validation loss: 0.042197	Best loss: 0.042197	Accuracy: 98.94%
7	Validation loss: 0.062623	Best loss: 0.042197	Accuracy: 98.44%
8	Validation loss: 0.052243	Best loss: 0.042197	Accuracy: 98.59%
9	Validation loss: 0.063239	Best loss: 0.042197	Accuracy: 98.40%
10	Validation loss: 0.049083	Best loss: 0.042197	Accuracy: 98.75%
11	Validation loss: 0.044761	Best loss: 0.042197	Accuracy: 98.94%
12	Validation loss: 0.035849	Best loss: 0.035849	Accuracy: 98.98%
13	Validation loss: 0.055390	Best loss: 0.035849	Accuracy: 98.79%
14	Validation loss: 0.040908	Best loss: 0.035849	Accuracy: 98.87%
15	Validation loss: 

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.2min remaining:    0.0s


0	Validation loss: 0.100687	Best loss: 0.100687	Accuracy: 97.50%
1	Validation loss: 0.055185	Best loss: 0.055185	Accuracy: 98.28%
2	Validation loss: 0.067285	Best loss: 0.055185	Accuracy: 98.05%
3	Validation loss: 0.051070	Best loss: 0.051070	Accuracy: 98.44%
4	Validation loss: 0.048451	Best loss: 0.048451	Accuracy: 98.59%
5	Validation loss: 0.057729	Best loss: 0.048451	Accuracy: 98.44%
6	Validation loss: 0.050405	Best loss: 0.048451	Accuracy: 98.87%
7	Validation loss: 0.047714	Best loss: 0.047714	Accuracy: 98.59%
8	Validation loss: 0.038666	Best loss: 0.038666	Accuracy: 98.94%
9	Validation loss: 0.045291	Best loss: 0.038666	Accuracy: 98.71%
10	Validation loss: 0.050645	Best loss: 0.038666	Accuracy: 98.87%
11	Validation loss: 0.036931	Best loss: 0.036931	Accuracy: 99.18%
12	Validation loss: 0.052774	Best loss: 0.036931	Accuracy: 98.79%
13	Validation loss: 0.057492	Best loss: 0.036931	Accuracy: 98.67%
14	Validation loss: 0.044714	Best loss: 0.036931	Accuracy: 98.79%
15	Validation loss: 

...

0	Validation loss: 0.124371	Best loss: 0.124371	Accuracy: 96.79%
1	Validation loss: 0.109740	Best loss: 0.109740	Accuracy: 96.87%
2	Validation loss: 0.077577	Best loss: 0.077577	Accuracy: 97.73%
3	Validation loss: 0.054508	Best loss: 0.054508	Accuracy: 98.55%
4	Validation loss: 0.061210	Best loss: 0.054508	Accuracy: 98.55%
5	Validation loss: 0.076453	Best loss: 0.054508	Accuracy: 97.93%
6	Validation loss: 0.054691	Best loss: 0.054508	Accuracy: 98.59%
7	Validation loss: 0.043732	Best loss: 0.043732	Accuracy: 98.63%
8	Validation loss: 0.049730	Best loss: 0.043732	Accuracy: 98.83%
9	Validation loss: 0.050617	Best loss: 0.043732	Accuracy: 98.55%
10	Validation loss: 0.061173	Best loss: 0.043732	Accuracy: 98.87%
11	Validation loss: 0.053836	Best loss: 0.043732	Accuracy: 98.71%
12	Validation loss: 0.059176	Best loss: 0.043732	Accuracy: 98.87%
13	Validation loss: 0.051977	Best loss: 0.043732	Accuracy: 98.87%
14	Validation loss: 0.078573	Best loss: 0.043732	Accuracy: 98.01%
15	Validation loss: 

0	Validation loss: 0.114591	Best loss: 0.114591	Accuracy: 97.22%
1	Validation loss: 0.069604	Best loss: 0.069604	Accuracy: 98.05%
2	Validation loss: 0.060601	Best loss: 0.060601	Accuracy: 98.08%
3	Validation loss: 0.062611	Best loss: 0.060601	Accuracy: 98.12%
4	Validation loss: 0.087278	Best loss: 0.060601	Accuracy: 97.73%
5	Validation loss: 0.081817	Best loss: 0.060601	Accuracy: 97.73%
6	Validation loss: 0.092783	Best loss: 0.060601	Accuracy: 97.89%
7	Validation loss: 0.146086	Best loss: 0.060601	Accuracy: 97.77%
8	Validation loss: 0.112748	Best loss: 0.060601	Accuracy: 97.97%
9	Validation loss: 0.075935	Best loss: 0.060601	Accuracy: 98.16%
10	Validation loss: 0.099613	Best loss: 0.060601	Accuracy: 98.40%
11	Validation loss: 0.030874	Best loss: 0.030874	Accuracy: 99.14%
12	Validation loss: 0.051259	Best loss: 0.030874	Accuracy: 98.63%
13	Validation loss: 0.096935	Best loss: 0.030874	Accuracy: 98.51%
14	Validation loss: 0.101482	Best loss: 0.030874	Accuracy: 98.40%
15	Validation loss: 

[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed: 285.0min finished


0	Validation loss: 0.057860	Best loss: 0.057860	Accuracy: 98.48%
1	Validation loss: 0.050512	Best loss: 0.050512	Accuracy: 98.44%
2	Validation loss: 0.062504	Best loss: 0.050512	Accuracy: 98.36%
3	Validation loss: 0.040096	Best loss: 0.040096	Accuracy: 98.87%
4	Validation loss: 0.042289	Best loss: 0.040096	Accuracy: 98.79%
5	Validation loss: 0.028066	Best loss: 0.028066	Accuracy: 99.10%
6	Validation loss: 0.041172	Best loss: 0.028066	Accuracy: 98.67%
7	Validation loss: 0.031058	Best loss: 0.028066	Accuracy: 99.10%
8	Validation loss: 0.042160	Best loss: 0.028066	Accuracy: 98.94%
9	Validation loss: 0.020907	Best loss: 0.020907	Accuracy: 99.37%
10	Validation loss: 0.033659	Best loss: 0.020907	Accuracy: 99.10%
11	Validation loss: 0.036124	Best loss: 0.020907	Accuracy: 98.87%
12	Validation loss: 0.022589	Best loss: 0.020907	Accuracy: 99.26%
13	Validation loss: 0.030218	Best loss: 0.020907	Accuracy: 98.98%
14	Validation loss: 0.029149	Best loss: 0.020907	Accuracy: 99.22%
15	Validation loss: 

RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=DNNClassifier(activation=<function elu at 0x7fade4683dd0>,
                                           batch_norm_momentum=None,
                                           batch_size=32, dropout_rate=None,
                                           initializer=<tensorflow.python.ops.init_ops.VarianceScaling object at 0x7fad6f31d8d0>,
                                           learning_rate=0.01,
                                           n_hidden_layers=5, n_neurons=100,
                                           optimizer_class=<class 'tensorflow.python.training.adam...
                                                       <function leaky_relu.<locals>.parametrized_leaky_relu at 0x7fad5893a680>,
                                                       <function leaky_relu.<locals>.parametrized_leaky_relu at 0x7fad5893a710>],
                                        'batch_norm_momentum': [0.9, 0.95, 0.98,
            

In [35]:
rnd_search_bn.best_params_

{'n_neurons': 160,
 'learning_rate': 0.01,
 'batch_size': 16,
 'batch_norm_momentum': 0.98,
 'activation': <function tensorflow.python.ops.gen_nn_ops.relu(features, name=None)>}

In [36]:
y_pred = rnd_search_bn.predict(X_test1)
accuracy_score(y_test1, y_pred)

0.993967697995719

**Wow 99.39% accuracy! Awesome!**

In [40]:
# Let's save this model
rnd_search_bn.best_estimator_.save("./best_mnist_model_0_to_4")

# Transfer Learning
**Exercise:** create a new DNN that reuses all the pretrained hidden layers of the previous model, freezes them, and replaces the softmax output layer with a new one.<br>
Let's load the model graph:

In [44]:
tf.reset_default_graph()

restore_saver = tf.train.import_meta_graph("./best_mnist_model_0_to_4.meta")

X = tf.get_default_graph().get_tensor_by_name("X:0")
y = tf.get_default_graph().get_tensor_by_name("y:0")
loss = tf.get_default_graph().get_tensor_by_name("loss:0")
Y_proba = tf.get_default_graph().get_tensor_by_name("Y_proba:0")
logits = Y_proba.op.inputs[0]
accuracy = tf.get_default_graph().get_tensor_by_name("accuracy:0")

In [46]:
tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)

[<tf.Variable 'hidden1/kernel:0' shape=(784, 160) dtype=float32_ref>,
 <tf.Variable 'hidden1/bias:0' shape=(160,) dtype=float32_ref>,
 <tf.Variable 'batch_normalization/gamma:0' shape=(160,) dtype=float32_ref>,
 <tf.Variable 'batch_normalization/beta:0' shape=(160,) dtype=float32_ref>,
 <tf.Variable 'hidden2/kernel:0' shape=(160, 160) dtype=float32_ref>,
 <tf.Variable 'hidden2/bias:0' shape=(160,) dtype=float32_ref>,
 <tf.Variable 'batch_normalization_1/gamma:0' shape=(160,) dtype=float32_ref>,
 <tf.Variable 'batch_normalization_1/beta:0' shape=(160,) dtype=float32_ref>,
 <tf.Variable 'hidden3/kernel:0' shape=(160, 160) dtype=float32_ref>,
 <tf.Variable 'hidden3/bias:0' shape=(160,) dtype=float32_ref>,
 <tf.Variable 'batch_normalization_2/gamma:0' shape=(160,) dtype=float32_ref>,
 <tf.Variable 'batch_normalization_2/beta:0' shape=(160,) dtype=float32_ref>,
 <tf.Variable 'hidden4/kernel:0' shape=(160, 160) dtype=float32_ref>,
 <tf.Variable 'hidden4/bias:0' shape=(160,) dtype=float32_ref

In [47]:
tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="logits")

[<tf.Variable 'logits/kernel:0' shape=(160, 5) dtype=float32_ref>,
 <tf.Variable 'logits/bias:0' shape=(5,) dtype=float32_ref>]

To freeze the lower layers, we will exclude their variables from the optimizer's list of trainable variables, keeping only the output layer's trainable variables:

In [48]:
learning_rate = 0.01

output_layer_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="logits")
optimizer = tf.train.AdamOptimizer(learning_rate, name="Adam2")
training_op = optimizer.minimize(loss, var_list=output_layer_vars)

In [49]:
correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

init = tf.global_variables_initializer()
five_frozen_saver = tf.train.Saver()

**Exercise:** train this new DNN on digits 5 to 9, using only 100 images per digit, and time how long it takes. Despite this small number of examples, can you achieve high precision.<br>
Let's create the training, validation and test sets. We need to subtract 5 from the labels because TensorFlow expects integers from 0 to n_classes-1.

In [50]:
X_train2_full = X_train[y_train >= 5]
y_train2_full = y_train[y_train >= 5] - 5
X_valid2_full = X_valid[y_valid >= 5]
y_valid2_full = y_valid[y_valid >= 5] - 5
X_test2 = X_test[y_test >= 5]
y_test2 = y_test[y_test >= 5] - 5

In [51]:
def sample_n_instances_per_class(X, y, n=100):
    Xs, ys = [], []
    for label in np.unique(y):
        idx = (y == label)
        Xc = X[idx][:n]
        yc = y[idx][:n]
        Xs.append(Xc)
        ys.append(yc)
    return np.concatenate(Xs), np.concatenate(ys)

X_train2, y_train2 = sample_n_instances_per_class(X_train2_full, y_train2_full, n=100)
X_valid2, y_valid2 = sample_n_instances_per_class(X_valid2_full, y_valid2_full, n=30)

Now let's train the model:

In [55]:
import time

n_epochs = 1000
batch_size = 16

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

with tf.Session() as sess:
    init.run()
    restore_saver.restore(sess, "./best_mnist_model_0_to_4")
    t0 = time.time()
    
    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(len(X_train2))
        for rnd_indices in np.array_split(rnd_idx, len(X_train2) // batch_size):
            X_batch, y_batch = X_train2[rnd_indices], y_train2[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={X: X_valid2, y: y_valid2})
        if loss_val < best_loss:
            save_path = five_frozen_saver.save(sess, "./my_mnist_model_5_to_9_five_frozen")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(epoch, loss_val, best_loss, acc_val * 100))
    
    t1 = time.time()
    print("Total training time: {:.1f}s".format(t1 - t0))
    
with tf.Session() as sess:
    five_frozen_saver.restore(sess, "./my_mnist_model_5_to_9_five_frozen")
    acc_test = accuracy.eval(feed_dict={X: X_test2, y: y_test2})
    print("Final test accuracy: {:.2f}%".format(acc_test * 100))

INFO:tensorflow:Restoring parameters from ./best_mnist_model_0_to_4
0	Validation loss: 1.933323	Best loss: 1.933323	Accuracy: 52.67%
1	Validation loss: 1.640835	Best loss: 1.640835	Accuracy: 58.67%
2	Validation loss: 1.475729	Best loss: 1.475729	Accuracy: 59.33%
3	Validation loss: 1.332690	Best loss: 1.332690	Accuracy: 59.33%
4	Validation loss: 1.240681	Best loss: 1.240681	Accuracy: 58.00%
5	Validation loss: 1.170182	Best loss: 1.170182	Accuracy: 59.33%
6	Validation loss: 1.106935	Best loss: 1.106935	Accuracy: 60.00%
7	Validation loss: 1.076100	Best loss: 1.076100	Accuracy: 61.33%
8	Validation loss: 1.040098	Best loss: 1.040098	Accuracy: 61.33%
9	Validation loss: 1.015161	Best loss: 1.015161	Accuracy: 62.67%
10	Validation loss: 1.000982	Best loss: 1.000982	Accuracy: 64.00%
11	Validation loss: 0.986794	Best loss: 0.986794	Accuracy: 64.67%
12	Validation loss: 0.978631	Best loss: 0.978631	Accuracy: 66.00%
13	Validation loss: 0.981341	Best loss: 0.978631	Accuracy: 65.33%
14	Validation loss

Well that's not a great accuracy, is it? Of course with such a tiny training set, and with only one layer to tweak, we should not expect miracles.

**Exercise:** try caching the frozen layers, and train the model again: how much faster is it now?
Let's start by getting a handle on the output of the last frozen layer:

In [56]:
hidden5_out = tf.get_default_graph().get_tensor_by_name("hidden5_out:0")

Now let's train the model using roughly the same code as earlier. The difference is that we compute the output of the top frozen layer at the beginning (both for the training set and the validation set), and we cache it. This makes training faster:

In [61]:
import time

n_epochs = 1000
batch_size = 16

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

with tf.Session() as sess:
    init.run()
    restore_saver.restore(sess, "./best_mnist_model_0_to_4")
    t0 = time.time()
    
    hidden5_train = hidden5_out.eval(feed_dict={X: X_train2, y: y_train2})
    hidden5_valid = hidden5_out.eval(feed_dict={X: X_valid2, y: y_valid2})
        
    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(len(X_train2))
        for rnd_indices in np.array_split(rnd_idx, len(X_train2) // batch_size):
            h5_batch, y_batch = hidden5_train[rnd_indices], y_train2[rnd_indices]
            sess.run(training_op, feed_dict={hidden5_out: h5_batch, y: y_batch})
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={hidden5_out: hidden5_valid, y: y_valid2})
        if loss_val < best_loss:
            save_path = five_frozen_saver.save(sess, "./my_mnist_model_5_to_9_five_frozen")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
            epoch, loss_val, best_loss, acc_val * 100))

    t1 = time.time()
    print("Total training time: {:.1f}s".format(t1 - t0))

with tf.Session() as sess:
    five_frozen_saver.restore(sess, "./my_mnist_model_5_to_9_five_frozen")
    acc_test = accuracy.eval(feed_dict={X: X_test2, y: y_test2})
    print("Final test accuracy: {:.2f}%".format(acc_test * 100))

INFO:tensorflow:Restoring parameters from ./best_mnist_model_0_to_4
0	Validation loss: 1.940089	Best loss: 1.940089	Accuracy: 51.33%
1	Validation loss: 1.649590	Best loss: 1.649590	Accuracy: 58.00%
2	Validation loss: 1.481291	Best loss: 1.481291	Accuracy: 60.00%
3	Validation loss: 1.351054	Best loss: 1.351054	Accuracy: 61.33%
4	Validation loss: 1.253872	Best loss: 1.253872	Accuracy: 58.67%
5	Validation loss: 1.177315	Best loss: 1.177315	Accuracy: 58.00%
6	Validation loss: 1.115052	Best loss: 1.115052	Accuracy: 60.67%
7	Validation loss: 1.076172	Best loss: 1.076172	Accuracy: 59.33%
8	Validation loss: 1.035310	Best loss: 1.035310	Accuracy: 61.33%
9	Validation loss: 1.014341	Best loss: 1.014341	Accuracy: 62.00%
10	Validation loss: 1.000529	Best loss: 1.000529	Accuracy: 62.67%
11	Validation loss: 0.985830	Best loss: 0.985830	Accuracy: 64.00%
12	Validation loss: 0.983637	Best loss: 0.983637	Accuracy: 62.67%
13	Validation loss: 0.976229	Best loss: 0.976229	Accuracy: 65.33%
14	Validation loss

**Exercise:** try again reusing just four hidden layers instead of five. Can you achieve a higher precision?<br>
Let's load the best model again, but this time we will create a new softmax output layer on top of the 4th hidden layer:


In [62]:
tf.reset_default_graph()

n_outputs = 5

restore_saver = tf.train.import_meta_graph("./best_mnist_model_0_to_4.meta")

X = tf.get_default_graph().get_tensor_by_name("X:0")
y = tf.get_default_graph().get_tensor_by_name("y:0")

hidden4_out = tf.get_default_graph().get_tensor_by_name("hidden4_out:0")
logits = tf.layers.dense(hidden4_out, n_outputs, kernel_initializer=he_init, name="new_logits")
Y_proba = tf.nn.softmax(logits)
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
loss = tf.reduce_mean(xentropy)
correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

And now let's create the training operation. We want to freeze all the layers except for the new output layer:

In [63]:
learning_rate = 0.01

output_layer_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="new_logits")
optimizer = tf.train.AdamOptimizer(learning_rate, name="Adam2")
training_op = optimizer.minimize(loss, var_list=output_layer_vars)

init = tf.global_variables_initializer()
four_frozen_saver = tf.train.Saver()

In [65]:
n_epochs = 1000
batch_size = 16

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

with tf.Session() as sess:
    init.run()
    restore_saver.restore(sess, "./best_mnist_model_0_to_4")
        
    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(len(X_train2))
        for rnd_indices in np.array_split(rnd_idx, len(X_train2) // batch_size):
            X_batch, y_batch = X_train2[rnd_indices], y_train2[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={X: X_valid2, y: y_valid2})
        if loss_val < best_loss:
            save_path = four_frozen_saver.save(sess, "./my_mnist_model_5_to_9_four_frozen")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
            epoch, loss_val, best_loss, acc_val * 100))

with tf.Session() as sess:
    four_frozen_saver.restore(sess, "./my_mnist_model_5_to_9_four_frozen")
    acc_test = accuracy.eval(feed_dict={X: X_test2, y: y_test2})
    print("Final test accuracy: {:.2f}%".format(acc_test * 100))

INFO:tensorflow:Restoring parameters from ./best_mnist_model_0_to_4
0	Validation loss: 1.025730	Best loss: 1.025730	Accuracy: 58.00%
1	Validation loss: 0.950150	Best loss: 0.950150	Accuracy: 61.33%
2	Validation loss: 0.909063	Best loss: 0.909063	Accuracy: 63.33%
3	Validation loss: 0.900198	Best loss: 0.900198	Accuracy: 62.67%
4	Validation loss: 0.900246	Best loss: 0.900198	Accuracy: 66.67%
5	Validation loss: 0.901613	Best loss: 0.900198	Accuracy: 64.67%
6	Validation loss: 0.876152	Best loss: 0.876152	Accuracy: 66.00%
7	Validation loss: 0.870765	Best loss: 0.870765	Accuracy: 66.00%
8	Validation loss: 0.903458	Best loss: 0.870765	Accuracy: 67.33%
9	Validation loss: 0.868697	Best loss: 0.868697	Accuracy: 64.00%
10	Validation loss: 0.882949	Best loss: 0.868697	Accuracy: 68.00%
11	Validation loss: 0.893347	Best loss: 0.868697	Accuracy: 67.33%
12	Validation loss: 0.900480	Best loss: 0.868697	Accuracy: 64.00%
13	Validation loss: 0.862914	Best loss: 0.862914	Accuracy: 63.33%
14	Validation loss

**Still not good!**

**Exercise:** now unfreeze the top two hidden layers and continue training: can you get the model to perform even better?

In [66]:
learning_rate = 0.01

unfrozen_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="hidden[34]|new_logits")
optimizer = tf.train.AdamOptimizer(learning_rate, name="Adam3")
training_op = optimizer.minimize(loss, var_list=unfrozen_vars)

init = tf.global_variables_initializer()
two_frozen_saver = tf.train.Saver()

In [67]:
n_epochs = 1000
batch_size = 16

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

with tf.Session() as sess:
    init.run()
    four_frozen_saver.restore(sess, "./my_mnist_model_5_to_9_four_frozen")
        
    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(len(X_train2))
        for rnd_indices in np.array_split(rnd_idx, len(X_train2) // batch_size):
            X_batch, y_batch = X_train2[rnd_indices], y_train2[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={X: X_valid2, y: y_valid2})
        if loss_val < best_loss:
            save_path = two_frozen_saver.save(sess, "./my_mnist_model_5_to_9_two_frozen")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
            epoch, loss_val, best_loss, acc_val * 100))

with tf.Session() as sess:
    two_frozen_saver.restore(sess, "./my_mnist_model_5_to_9_two_frozen")
    acc_test = accuracy.eval(feed_dict={X: X_test2, y: y_test2})
    print("Final test accuracy: {:.2f}%".format(acc_test * 100))

INFO:tensorflow:Restoring parameters from ./my_mnist_model_5_to_9_four_frozen
0	Validation loss: 0.882573	Best loss: 0.882573	Accuracy: 64.00%
1	Validation loss: 0.803291	Best loss: 0.803291	Accuracy: 69.33%
2	Validation loss: 0.814970	Best loss: 0.803291	Accuracy: 68.67%
3	Validation loss: 0.891448	Best loss: 0.803291	Accuracy: 68.67%
4	Validation loss: 0.817812	Best loss: 0.803291	Accuracy: 70.00%
5	Validation loss: 0.787817	Best loss: 0.787817	Accuracy: 72.00%
6	Validation loss: 0.810686	Best loss: 0.787817	Accuracy: 72.67%
7	Validation loss: 0.746961	Best loss: 0.746961	Accuracy: 74.67%
8	Validation loss: 0.814620	Best loss: 0.746961	Accuracy: 71.33%
9	Validation loss: 0.770731	Best loss: 0.746961	Accuracy: 74.00%
10	Validation loss: 0.696622	Best loss: 0.696622	Accuracy: 80.00%
11	Validation loss: 0.709340	Best loss: 0.696622	Accuracy: 78.67%
12	Validation loss: 0.701227	Best loss: 0.696622	Accuracy: 77.33%
13	Validation loss: 0.660000	Best loss: 0.660000	Accuracy: 81.33%
14	Valid

Let's check what accuracy we can get by unfreezing all layers:

In [68]:
learning_rate = 0.01

optimizer = tf.train.AdamOptimizer(learning_rate, name="Adam4")
training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()
no_frozen_saver = tf.train.Saver()

In [69]:
n_epochs = 1000
batch_size = 20

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

with tf.Session() as sess:
    init.run()
    two_frozen_saver.restore(sess, "./my_mnist_model_5_to_9_two_frozen")
        
    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(len(X_train2))
        for rnd_indices in np.array_split(rnd_idx, len(X_train2) // batch_size):
            X_batch, y_batch = X_train2[rnd_indices], y_train2[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={X: X_valid2, y: y_valid2})
        if loss_val < best_loss:
            save_path = no_frozen_saver.save(sess, "./my_mnist_model_5_to_9_no_frozen")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
            epoch, loss_val, best_loss, acc_val * 100))

with tf.Session() as sess:
    no_frozen_saver.restore(sess, "./my_mnist_model_5_to_9_no_frozen")
    acc_test = accuracy.eval(feed_dict={X: X_test2, y: y_test2})
    print("Final test accuracy: {:.2f}%".format(acc_test * 100))

INFO:tensorflow:Restoring parameters from ./my_mnist_model_5_to_9_two_frozen
0	Validation loss: 0.454268	Best loss: 0.454268	Accuracy: 88.67%
1	Validation loss: 0.407702	Best loss: 0.407702	Accuracy: 90.00%
2	Validation loss: 0.321115	Best loss: 0.321115	Accuracy: 90.00%
3	Validation loss: 0.396031	Best loss: 0.321115	Accuracy: 88.67%
4	Validation loss: 0.296740	Best loss: 0.296740	Accuracy: 91.33%
5	Validation loss: 0.330068	Best loss: 0.296740	Accuracy: 91.33%
6	Validation loss: 0.324563	Best loss: 0.296740	Accuracy: 92.00%
7	Validation loss: 0.327529	Best loss: 0.296740	Accuracy: 92.00%
8	Validation loss: 0.322496	Best loss: 0.296740	Accuracy: 92.00%
9	Validation loss: 0.334411	Best loss: 0.296740	Accuracy: 91.33%
10	Validation loss: 0.328087	Best loss: 0.296740	Accuracy: 91.33%
11	Validation loss: 0.328466	Best loss: 0.296740	Accuracy: 91.33%
12	Validation loss: 0.330855	Best loss: 0.296740	Accuracy: 91.33%
13	Validation loss: 0.328014	Best loss: 0.296740	Accuracy: 92.00%
14	Valida



Let's compare that to a DNN trained from scratch:


In [70]:
dnn_clf_5_to_9 = DNNClassifier(n_hidden_layers=4, random_state=42)
dnn_clf_5_to_9.fit(X_train2, y_train2, n_epochs=1000, X_valid=X_valid2, y_valid=y_valid2)

0	Validation loss: 0.724673	Best loss: 0.724673	Accuracy: 66.67%
1	Validation loss: 0.528852	Best loss: 0.528852	Accuracy: 88.67%
2	Validation loss: 0.538593	Best loss: 0.528852	Accuracy: 87.33%
3	Validation loss: 0.587227	Best loss: 0.528852	Accuracy: 88.67%
4	Validation loss: 0.516730	Best loss: 0.516730	Accuracy: 86.67%
5	Validation loss: 0.460854	Best loss: 0.460854	Accuracy: 88.00%
6	Validation loss: 0.542612	Best loss: 0.460854	Accuracy: 88.00%
7	Validation loss: 0.504381	Best loss: 0.460854	Accuracy: 88.00%
8	Validation loss: 0.654456	Best loss: 0.460854	Accuracy: 88.67%
9	Validation loss: 0.727307	Best loss: 0.460854	Accuracy: 88.00%
10	Validation loss: 0.538752	Best loss: 0.460854	Accuracy: 93.33%
11	Validation loss: 0.589845	Best loss: 0.460854	Accuracy: 90.67%
12	Validation loss: 0.440367	Best loss: 0.440367	Accuracy: 92.67%
13	Validation loss: 0.456235	Best loss: 0.440367	Accuracy: 92.00%
14	Validation loss: 0.677045	Best loss: 0.440367	Accuracy: 92.67%
15	Validation loss: 

DNNClassifier(activation=<function elu at 0x7fade4683dd0>,
              batch_norm_momentum=None, batch_size=32, dropout_rate=None,
              initializer=<tensorflow.python.ops.init_ops.VarianceScaling object at 0x7fad6f31d8d0>,
              learning_rate=0.01, n_hidden_layers=4, n_neurons=100,
              optimizer_class=<class 'tensorflow.python.training.adam.AdamOptimizer'>,
              random_state=42)

In [71]:
y_pred = dnn_clf_5_to_9.predict(X_test2)
accuracy_score(y_test2, y_pred)

0.9024891997531372

Here due to less training data transfer learning is failing, we need to do osme tweaks to it to make it more accurate.